In [333]:
import numpy as np
from qiskit.quantum_info.operators import Operator
from qiskit.circuit.library.standard_gates import (
    IGate,
    RXGate,
    RYGate,
    RZGate,
    CZGate,
    CXGate,
    SwapGate,
    SXGate,
    CRZGate,
    HGate,
    RXXGate,
    RYYGate,
    RZZGate
)
from qiskit import QuantumCircuit

In [290]:
#use Operator.equiv since returns true if equivalent up to a global phase
print(Operator(RXGate(np.pi/2).to_matrix()).equiv(Operator(SXGate().to_matrix())))
print(SXGate().to_matrix() == RXGate(np.pi/2).to_matrix())

True
[[False False]
 [False False]]


In [293]:
Utarget = RZGate(np.pi/4)
Uk = RXGate(np.pi/2).to_matrix()
Uy = RYGate(-np.pi/4).to_matrix()
#Utarget = SwapGate()
d = Utarget.decompositions
for c in d:
    print(c.draw())
    print(Operator(c).data)

d = Utarget.definition
for c in d:
    print(c)

global phase: 15π/8
   ┌─────────┐
q: ┤ U1(π/4) ├
   └─────────┘
[[0.92387953-0.38268343j 0.        +0.j        ]
 [0.        +0.j         0.92387953+0.38268343j]]
   ┌────┐┌──────────┐┌──────┐
q: ┤ √X ├┤ Ry(-π/4) ├┤ √Xdg ├
   └────┘└──────────┘└──────┘
[[0.92387953-0.38268343j 0.        +0.j        ]
 [0.        +0.j         0.92387953+0.38268343j]]
(Instruction(name='u1', num_qubits=1, num_clbits=0, params=[0.7853981633974483]), [Qubit(QuantumRegister(1, 'q'), 0)], [])


In [294]:
Ufix  = SXGate().inverse().to_matrix() @ RYGate(-np.pi/4).to_matrix() @ SXGate().to_matrix()
print(Ufix)
print(Operator(Utarget).equiv(Operator(Ufix)))

[[0.92387953-0.38268343j 0.        +0.j        ]
 [0.        +0.j         0.92387953+0.38268343j]]
True


In [296]:
Ufixit = np.eye(4)

qc = QuantumCircuit(2)
qc.append(RZGate(np.pi/8), [1])
Ufixit = Operator(qc).data @ Ufixit

qc = QuantumCircuit(2)
qc.append(HGate(), [1])
Ufixit = Operator(qc).data @ Ufixit

qc = QuantumCircuit(2)
qc.append(CZGate(), [0,1])
Ufixit = Operator(qc).data @ Ufixit

qc = QuantumCircuit(2)
qc.append(RXGate(-np.pi/8), [1])
Ufixit = Operator(qc).data @ Ufixit

qc = QuantumCircuit(2)
qc.append(CZGate(), [0,1])
Ufixit = Operator(qc).data @ Ufixit

qc = QuantumCircuit(2)
qc.append(HGate(), [1])
Ufixit = Operator(qc).data @ Ufixit

In [298]:
#TODO, initialize an extended version of basis with gates
basis = [RZGate, HGate, RXGate, CZGate]
extended_basis = [RZGate(np.pi/8), RZGate(-np.pi/8), RXGate(np.pi/8), RXGate(-np.pi/8), HGate(), CZGate()]

In [299]:
def build_stack(target_basis):
    gate_list = []
    for gate in target_basis:
        if gate.num_qubits == 1:
            gate_list.append((gate, [0]))
            gate_list.append((gate, [1]))
        else:
            gate_list.append((gate, [0,1]))
            gate_list.append((gate, [1,0]))
    return gate_list

In [300]:
class Node():
    def __init__(self, parent, gate):
        self.gate = gate
        self.parent = parent
        self.children = []
        self.depth = 0
    
    def build_children(self, target_basis):
        for gate in build_stack(target_basis):
            self.children.append(Node(self, gate))
    
    def set_value(self, value):
        self.value = value
    
    def to_circuit(self):
        gate_list = []
        current_node = sol
        while current_node.gate != None:
            gate_list.append(current_node.gate)
            current_node = current_node.parent
        qc = QuantumCircuit(2)
        for gate in reversed(gate_list):
            qc.append(gate[0], gate[1])
        return qc

class SearchTree():
    def __init__(self):
        self.root = Node(None, None)
        self.root.set_value(np.eye(4))

In [321]:
def BFS(target, target_basis):
    tree = SearchTree()
    queue = [tree.root]
    depth_search = []
    while 1:

        current_node = queue.pop(0)
        current_node.build_children(target_basis)
        queue.extend(current_node.children)

        if current_node.depth not in depth_search:
            print(f"Now searching circuits of depth {current_node.depth}")
            depth_search.append(current_node.depth)

        for child in current_node.children:

            qc = QuantumCircuit(2)
            qc.append(child.gate[0], child.gate[1])
            child.set_value(Operator(qc).data @ current_node.value)
            child.depth = current_node.depth + 1

            if Operator(target).equiv(child.value):
                return child

In [322]:
sol = BFS(target = CRZGate(np.pi/4), target_basis = [CRZGate(np.pi/4)])
sol.to_circuit().draw()

Now searching circuits of depth 0


q_0: ─────■─────
     ┌────┴────┐
q_1: ┤ Rz(π/4) ├
     └─────────┘

In [323]:
sol = BFS(target = CRZGate(np.pi/4), target_basis = [RZGate(np.pi/8), CXGate(), RZGate(-np.pi/8)])
sol.to_circuit().draw()

Now searching circuits of depth 0
Now searching circuits of depth 1
Now searching circuits of depth 2
Now searching circuits of depth 3


q_0: ─────────────■────────────────■──
     ┌─────────┐┌─┴─┐┌──────────┐┌─┴─┐
q_1: ┤ Rz(π/8) ├┤ X ├┤ Rz(-π/8) ├┤ X ├
     └─────────┘└───┘└──────────┘└───┘

In [305]:
sol = BFS(target = CRZGate(np.pi/4), target_basis = [RZGate(np.pi/8), RXGate(-np.pi/8), HGate(), CZGate()])
sol.to_circuit().draw()

Now searching circuits of depth 0
Now searching circuits of depth 1
Now searching circuits of depth 2
Now searching circuits of depth 3
Now searching circuits of depth 4
Now searching circuits of depth 5


┌───┐      ┌──────────┐   ┌───┐
q_0: ───┤ H ├────■─┤ Rx(-π/8) ├─■─┤ H ├
     ┌──┴───┴──┐ │ └──────────┘ │ └───┘
q_1: ┤ Rz(π/8) ├─■──────────────■──────
     └─────────┘

Note that for this basis, Qiskit Transpiler finds a slightly different circuit

In [315]:
qc = QuantumCircuit(2)
qc.append(RZGate(np.pi/8), [1])
qc.append(HGate(), [1])
qc.append(CZGate(), [0,1])
qc.append(RXGate(-np.pi/8), [1])
qc.append(CZGate(), [0,1])
qc.append(HGate(), [1])
print(qc.draw())

                                            
q_0: ─────────────────■──────────────■──────
     ┌─────────┐┌───┐ │ ┌──────────┐ │ ┌───┐
q_1: ┤ Rz(π/8) ├┤ H ├─■─┤ Rx(-π/8) ├─■─┤ H ├
     └─────────┘└───┘   └──────────┘   └───┘


In [317]:
from qiskit.quantum_info import Statevector
Statevector.from_instruction(qc).equiv(Statevector.from_instruction(sol.to_circuit()))

True

In [331]:
sol = BFS(target = CRZGate(np.pi/4), target_basis = [CZGate(), RXGate(np.pi/2), RXGate(-np.pi/2), RXGate(np.pi), RYGate(-np.pi/4), RYGate(np.pi/2)])
sol.to_circuit().draw()

Now searching circuits of depth 0
Now searching circuits of depth 1
Now searching circuits of depth 2
Now searching circuits of depth 3
Now searching circuits of depth 4
Now searching circuits of depth 5
Now searching circuits of depth 6


KeyboardInterrupt: 

In [337]:
theta = np.pi/2
sol = BFS(target = SwapGate(), 
target_basis = [RXXGate(theta), RXXGate(-theta), RYYGate(theta), RYYGate(-theta), RZZGate(theta), RZZGate(theta)])
sol.to_circuit().draw()

Now searching circuits of depth 0
Now searching circuits of depth 1
Now searching circuits of depth 2


┌───────────┐┌───────────┐          
q_0: ┤0          ├┤0          ├─■────────
     │  Rxx(π/2) ││  Ryy(π/2) │ │ZZ(π/2) 
q_1: ┤1          ├┤1          ├─■────────
     └───────────┘└───────────┘

In [336]:
from qiskit import transpile
qc = QuantumCircuit(2)
qc.swap(0,1)
transpile(qc, basis_gates= ['rxx', 'ryy', 'rzz'])

TranspilerError: "Unable to map source basis {('swap', 2)} to target basis {'delay', 'reset', 'barrier', 'snapshot', 'rxx', 'rzz', 'measure', 'ryy'} over library <qiskit.circuit.equivalence.EquivalenceLibrary object at 0x7f9cd244d460>."